<h2>3:00:00 / 3:11:52</h2>

<h1>Workflow</h1>

<pre>
    0. Import important liberaries
    1. Get data ready (turn into tensors and batches)
    2. Build a logistic regression model
    3. Pick loss function and optimizer
    4. Build a training loop
    5. Evaluate your model
    6. How to improve our model
    7. Save your model
</pre>

<h2>0. Import important liberaries</h2>

In [1]:
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

<h2>1. Get data ready (turn into tensors and batches</h2>

<h3>Download datasets</h3>

In [2]:
train_dataset = datasets.FashionMNIST(root="/dataset", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.FashionMNIST(root="/dataset", train=False, transform=transforms.ToTensor())

In [3]:
train_dataset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: /dataset
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
test_dataset

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: /dataset
    Split: Test
    StandardTransform
Transform: ToTensor()

<h2>2. Convert data into batches</h2>

In [5]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
len(train_loader) # 60000 / 64 = 938

938

In [7]:
len(test_loader) #10000 / 64 = 157

157

<h2>3. Build logistic model</h2>

In [8]:
class LogisticRegression(nn.Module):
    def __init__(self, in_dim, n_class):
        super().__init__()
        self.linear = nn.Linear(in_dim, n_class)

    def forward(self, x):
        out = self.linear(x)
        return out

In [9]:
model = LogisticRegression(28*28, 10) #28*28 = 784

In [10]:
model

LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)

In [11]:
model.state_dict()

OrderedDict([('linear.weight',
              tensor([[-0.0353, -0.0342, -0.0228,  ...,  0.0010, -0.0133,  0.0145],
                      [-0.0303, -0.0276, -0.0166,  ...,  0.0224,  0.0346, -0.0339],
                      [-0.0169,  0.0106,  0.0339,  ..., -0.0219, -0.0163, -0.0336],
                      ...,
                      [-0.0251, -0.0162, -0.0070,  ...,  0.0195, -0.0356, -0.0066],
                      [-0.0199,  0.0050, -0.0245,  ...,  0.0321,  0.0023, -0.0148],
                      [-0.0329,  0.0018, -0.0279,  ...,  0.0064,  0.0002, -0.0127]])),
             ('linear.bias',
              tensor([ 0.0065,  0.0031,  0.0255,  0.0246, -0.0024, -0.0156, -0.0190,  0.0004,
                      -0.0280,  0.0325]))])

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Code for GPU

In [13]:
device

'cuda'

In [14]:
model = model.to(device) #Send model to tun at GPU

<h2>4. Pick a loss function and optimizer</h2>

In [15]:
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [16]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0353, -0.0342, -0.0228,  ...,  0.0010, -0.0133,  0.0145],
         [-0.0303, -0.0276, -0.0166,  ...,  0.0224,  0.0346, -0.0339],
         [-0.0169,  0.0106,  0.0339,  ..., -0.0219, -0.0163, -0.0336],
         ...,
         [-0.0251, -0.0162, -0.0070,  ...,  0.0195, -0.0356, -0.0066],
         [-0.0199,  0.0050, -0.0245,  ...,  0.0321,  0.0023, -0.0148],
         [-0.0329,  0.0018, -0.0279,  ...,  0.0064,  0.0002, -0.0127]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([ 0.0065,  0.0031,  0.0255,  0.0246, -0.0024, -0.0156, -0.0190,  0.0004,
         -0.0280,  0.0325], device='cuda:0', requires_grad=True)]

In [17]:
model.state_dict()

OrderedDict([('linear.weight',
              tensor([[-0.0353, -0.0342, -0.0228,  ...,  0.0010, -0.0133,  0.0145],
                      [-0.0303, -0.0276, -0.0166,  ...,  0.0224,  0.0346, -0.0339],
                      [-0.0169,  0.0106,  0.0339,  ..., -0.0219, -0.0163, -0.0336],
                      ...,
                      [-0.0251, -0.0162, -0.0070,  ...,  0.0195, -0.0356, -0.0066],
                      [-0.0199,  0.0050, -0.0245,  ...,  0.0321,  0.0023, -0.0148],
                      [-0.0329,  0.0018, -0.0279,  ...,  0.0064,  0.0002, -0.0127]],
                     device='cuda:0')),
             ('linear.bias',
              tensor([ 0.0065,  0.0031,  0.0255,  0.0246, -0.0024, -0.0156, -0.0190,  0.0004,
                      -0.0280,  0.0325], device='cuda:0'))])

<h2>5. Building a training loop</h2>

In [18]:
num_epochs = 8
for epoch in range(num_epochs):
    print('* ' *24)
    print(f'epoch {epoch +1}')
    since = time.time()
    running_loss = 0.0
    running_accu = 0.0
    model.train()
    for i, data in enumerate(train_loader, 1): #937
        img, label = data
        img = img.view(img.size(0),-1)
        img = img.to(device)
        label = label.to(device)
        out = model(img) #forward pass
        loss = criterion(out, label)
        running_loss += loss.item()
        _,pred = torch.max(out, 1)
        running_accu += (pred == label).float().mean()
        optimizer.zero_grad()
        loss.backward() #backward pass
        optimizer.step()
        if i % 300 == 0:
            print(f'[{epoch+1} / {num_epochs}], Loss: {running_loss / i:.6f}, Accuracy: {running_accu/i:.6f}')
    print(f'Finish {epoch+1} Epoch, Loss: {running_loss/i:.6f}, Accu: {running_accu/i:.6f}')

    
    """6. Evaluate your model"""
    model.eval()
    eval_loss = 0.
    eval_accu = 0.
    for data in test_loader:
        img,label = data
        img = img.view(img.size(0),-1)
        img = img.to(device)
        label = label.to(device)
        with torch.no_grad():
            out = model(img)
            loss = criterion(out,label)
        eval_loss += loss.item()
        _,pred = torch.max(out, 1)
        eval_accu += (pred == label).float().mean()
    print()
    
    print(f'Test loss: {eval_loss / len(test_loader):.6f}, Accu: {eval_accu / len(train_loader):.6f}')
    print(f'Time:{(time.time() - since):.1f} s')


* * * * * * * * * * * * * * * * * * * * * * * * 
epoch 1
[1 / 8], Loss: 2.026895, Accuracy: 0.428333
[1 / 8], Loss: 1.838077, Accuracy: 0.525443
[1 / 8], Loss: 1.705580, Accuracy: 0.565764
Finish 1 Epoch, loss: 1.691357, Accu: 0.569513

Test loss: 1.349159, Accu: 0.108642
Time:23.1 s
* * * * * * * * * * * * * * * * * * * * * * * * 
epoch 2
[2 / 8], Loss: 1.285837, Accuracy: 0.659479
[2 / 8], Loss: 1.236313, Accuracy: 0.663672
[2 / 8], Loss: 1.197760, Accuracy: 0.665694
Finish 2 Epoch, loss: 1.194135, Accu: 0.665595

Test loss: 1.097152, Accu: 0.111524
Time:22.7 s
* * * * * * * * * * * * * * * * * * * * * * * * 
epoch 3
[3 / 8], Loss: 1.067548, Accuracy: 0.677865
[3 / 8], Loss: 1.045023, Accuracy: 0.678750
[3 / 8], Loss: 1.023707, Accuracy: 0.684635
Finish 3 Epoch, loss: 1.020899, Accu: 0.685185

Test loss: 0.980327, Accu: 0.114256
Time:23.0 s
* * * * * * * * * * * * * * * * * * * * * * * * 
epoch 4
[4 / 8], Loss: 0.953486, Accuracy: 0.697135
[4 / 8], Loss: 0.942703, Accuracy: 0.700911
